<h2> Load Data</h2>

In [1]:
import os
import warnings

import ipynb.fs.full.GenSnippetsLib as snippet
import matplotlib.image as mpimg

%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import preparers
import seaborn as sns
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
data = preparers.load_queried(
    participant_number=1,
    snippets=["BogoSort", "Ackerman"],
    query_code=True,
    query_eeg=True,
    query_eye_tracking=True,
    query_behavioral=True,
    query_log=True,
    query_meta=True,
)

Loading Ackerman Files ......
Loading BogoSort Files ......


<h2>potential funtion arguments</h2>

In [ ]:
fig, ax = (None, None)
display_width = 1920
display_height = 1080
image_path = "../CodeSnippets/Generators/" + "BogoSort" + ".json"
img, result = snippet.create_image(image_path, font_path="/../fonts/ttf/")
bg = snippet.create_background(display_width, display_height, (200, 200, 200, 255))
img = snippet.place_image_on(bg, np.array(img), 0.5, 0.5)

<h2> Prepare queried Data</h2>

In [5]:
df_eyetracking = data["BogoSort"]["Code"]["EyeTracking"][
    [
        "l_valid",
        "r_valid",
        "l_display_x",
        "l_display_y",
        "r_display_x",
        "r_display_y",
        "time",
        "l_pupil_diameter",
        "r_pupil_diameter",
    ]
]
df_eyetracking["l_display_x"] = df_eyetracking["l_display_x"].apply(lambda x: x * display_width)
df_eyetracking["l_display_x"] = df_eyetracking["l_display_x"].astype(int)

df_eyetracking["r_display_x"] = df_eyetracking["r_display_x"].apply(lambda x: x * display_width)
df_eyetracking["r_display_x"] = df_eyetracking["r_display_x"].astype(int)

df_eyetracking["l_display_y"] = df_eyetracking["l_display_y"].apply(lambda x: x * display_height)
df_eyetracking["l_display_y"] = df_eyetracking["l_display_y"].astype(int)

df_eyetracking["r_display_y"] = df_eyetracking["r_display_y"].apply(lambda x: x * display_height)
df_eyetracking["r_display_y"] = df_eyetracking["r_display_y"].astype(int)

df_eyetracking["time"] = df_eyetracking["time"] - df_eyetracking["time"][0]

<h2> Make Heatmaps</h2>

In [6]:
def create_circular_mask(h, w, center=None, radius=None):

    if center is None:  # use the middle of the image
        center = (int(w / 2), int(h / 2))
    if radius is None:  # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w - center[0], h - center[1])

    Y, X = np.ogrid[:h, :w]
    x_low = max(center[0] - radius, 0.0)
    x_high = min(center[0] + radius, w)
    y_low = max(center[1] - radius, 0.0)
    y_high = min(center[1] + radius, w)
    X = X[:, x_low:x_high]
    Y = Y[y_low:y_high]
    dist_from_center = np.full([h, w], radius + 1)
    dist_from_center[y_low:y_high, x_low:x_high] = np.sqrt((X - center[0]) ** 2 + (Y - center[1]) ** 2)

    mask = dist_from_center <= radius
    return mask


def create_heapmap(eye_x, eye_y, eye_valid, d_time, h, w, radius=30):
    hmap = np.full([h, w], 0.0)
    heat = np.full([h, w], d_time)
    for idx in range(len(eye_x)):
        if eye_valid[idx]:
            x = eye_x[idx]
            y = eye_y[idx]
            mask = create_circular_mask(h, w, (x, y), radius)
            hmap[mask] += heat[mask]
    return hmap

In [ ]:
def apply_filter(data, kernel_width):
    print(data)


apply_filter(df_edf_eyetrackingking)

In [7]:
data_right = create_heapmap(
    df_eyetracking["r_display_x"],
    df_eyetracking["r_display_y"],
    df_eyetracking["r_valid"],
    0.004,
    display_height,
    display_width,
)
data_left = create_heapmap(
    df_eyetracking["l_display_x"],
    df_eyetracking["l_display_y"],
    df_eyetracking["l_valid"],
    0.004,
    display_height,
    display_width,
)

data_left = data_left / data_left.max()
data_right = data_right / data_right.max()

left_color = (1.0, 0.0, 0.0)
right_color = (0.0, 1.0, 0.0)

data_right = np.array(
    [data_right * right_color[0], data_right * right_color[1], data_right * right_color[2], data_right]
)
data_right = np.moveaxis(data_right, 0, 2)

data_left = np.array([data_left * left_color[0], data_left * left_color[1], data_left * left_color[2], data_left])
data_left = np.moveaxis(data_left, 0, 2)

In [36]:
fig, ax = plt.subplots()
ax.imshow(data_right, zorder=2, alpha=0.5)
ax.imshow(img, zorder=1)
plt.show()

In [38]:
fig, ax = plt.subplots()
ax.imshow(data_left, zorder=2, alpha=0.5)
ax.imshow(img, zorder=1)
plt.show()

In [40]:
fig, ax = plt.subplots()
ax.imshow(data_left, zorder=2, alpha=0.5)
ax.imshow(data_right, zorder=3, alpha=0.5)
ax.imshow(img, zorder=1)
plt.show()

<h2>Make Sequence Diagrams</h2>

In [63]:
def create_sequence_diagram(eye_y, eye_valid, h, w, offset=0, step=0.1):
    sequence = np.full([h, w], 0.0)
    current_x = offset
    current = None
    x_step = step

    for idx, y in enumerate(eye_y):
        if eye_valid[idx]:
            current = (min(max(0, y), h - 1), current_x)
            break

    for idx, y in enumerate(eye_y):
        if eye_valid[idx]:
            old_x = current[1]
            old_y = current[0]
            next_y = min(max(0, y), h - 1)
            next_x = min(max(0, current[1] + x_step), w - 1)
            sequence[old_y, int(old_x) : int(next_x)] = 1.0
            sequence[old_y:next_y, int(next_x)] = 1.0
            sequence[next_y:old_y, int(next_x)] = 1.0
            current = (next_y, next_x)

    return sequence

In [73]:
data_right = create_sequence_diagram(
    df_eyetracking["r_display_x"],
    df_eyetracking["r_display_y"][0],
    df_eyetracking["r_valid"][0],
    display_height,
    display_width,
    offset=670,
)
data_right = np.array(
    [data_right * right_color[0], data_right * right_color[1], data_right * right_color[2], data_right]
)
data_right = np.moveaxis(data_right, 0, 2)

data_left = create_sequence_diagram(
    df_eyetracking["l_display_y"], df_eyetracking["l_valid"], display_height, display_width, offset=670
)
data_left = np.array([data_left * left_color[0], data_left * left_color[1], data_left * left_color[2], data_left])
data_left = np.moveaxis(data_left, 0, 2)

In [76]:
fig, ax = plt.subplots()
ax.imshow(data_left, zorder=2, alpha=0.5)
ax.imshow(data_right, zorder=3, alpha=0.5)
ax.imshow(img, zorder=1)
plt.show()
data_right.shape

(1080, 1920, 4)

<h2> Make marker</h2>

In [95]:
def create_marker(eye_x, eye_y, eye_valid, h, w, radius=30):
    if not eye_valid:
        return np.full([h, w], 0.0)
    circle_1 = create_circular_mask(h, w, (eye_x, eye_y), radius)
    circle_2 = create_circular_mask(h, w, (eye_x, eye_y), radius - 3)
    circle_1 = circle_1 * 1.0
    circle_2 = circle_2 * 1.0
    return circle_1 - circle_2


nmb = 0
data_right = create_marker(
    df_eyetracking["r_display_x"][nmb],
    df_eyetracking["r_display_y"][nmb],
    df_eyetracking["r_valid"][nmb],
    display_height,
    display_width,
)
data_right = np.array(
    [data_right * right_color[0], data_right * right_color[1], data_right * right_color[2], data_right]
)
data_right = np.moveaxis(data_right, 0, 2)
plt.imshow(data_right)

In [81]:
fig, ax = plt.subplots()
ax.imshow(data_left, zorder=2, alpha=0.5)
ax.imshow(data_right, zorder=3, alpha=0.5)
ax.imshow(img, zorder=1)
plt.show()
data_right.shape

317